## This code is written by A. Aysu Çatal (a.aysucatal@gmail.com) to predict Af temperature and thermal hysteresis of NiTiHf alloy with literature data.
## August 2020
## If you want to adopt this code or our dataset to your project, you can refer to our paper "Design of a NiTiHf shape memory alloy with an austenite finish temperature beyond 400 °C utilizing artificial intelligence", accepted for publication in the Journal of Alloys and Compounds

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import normalize
from sklearn.pipeline import Pipeline
import matplotlib.pyplot as plt
from matplotlib import style
from pandas import read_csv
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.preprocessing import PowerTransformer
import scipy.stats
from scipy.stats.stats import pearsonr
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model

from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import initializers
from keras.layers import Dense
from keras import regularizers, losses

In [ ]:
data = pd.read_csv("dataset_original.csv", header=0, index_col=False ,sep=';', decimal='.')

In [ ]:
data

In [ ]:
pearson_matrix = data.corr()
pearson_matrix = pearson_matrix.values
feature_names = list(data.columns)

In [ ]:
fig, ax = plt.subplots()
im = ax.imshow(pearson_matrix, cmap='seismic', interpolation='nearest')
ax.set_xticks(np.arange(len(pearson_matrix)))
ax.set_yticks(np.arange(len(pearson_matrix)))
plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
         rotation_mode="anchor")
ax.set_xticklabels(feature_names)
ax.set_yticklabels(feature_names)
plt.show()

## At this point, we exported and examined pearson matrix and rearranged the features' order for further implications of feature subset selection

In [ ]:
Ni = data["Ni"]
Ti =  data["Ti"]
Hf = data["Hf"]
Cv = data["Cv"]
melt = data["melting point"]
elec = data["electronegativity"]
ar = data["atomic radius"]
ir = data["ionic radius"]
mass = data["atomic mass"]
wcr = data["wcr"]
hom_T = data["hom.T"]
hom_time = data["hom.time"]
sht_T = data["sht.T"]
sht_time = data["sht.time"]
ag_T = data["ag.T"]
ag_time = data["ag.time"]
Af = data["Af"]
Hyst = data["Hysteresis"]

In [ ]:
# Dataset is rearranged according to correlations: 
# (feature with higher correlations are put at the end)
data_rac = pd.concat([Ni, Ti, hom_T, hom_time, sht_T, sht_time, ag_T, ag_time, 
                      ar, elec, wcr, Cv, ir, mass, melt, Hf, Af, Hyst], axis=1) 
data_rac.head(5) #

In [ ]:
Af_dataset = data_rac.drop(columns=['Hysteresis'], inplace=False)
Hyst_dataset = data_rac.drop(columns=['Af'], inplace=False)

# Normalization and Transformation:

In [ ]:
inputs_un = Af_dataset.iloc[:,0:-1]
targets = np.sqrt(Af_dataset.iloc[:,-1])
input_headers = list(inputs_un.columns)

In [ ]:
scaler = MinMaxScaler()
inputs = scaler.fit_transform(inputs_un)
inputs = pd.DataFrame(inputs, columns = input_headers)

In [ ]:
plt.boxplot(inputs)
plt.show()

In [ ]:
plt.hist(targets)
plt.show()

# NN model

In [ ]:
inputs = np.array(inputs)  #according to feature subset selection number of columns should be changed
n_rows, n_feature = np.shape(inputs)

In [ ]:
# splitting test and train data:
X_train, X_test, Y_train, Y_test = train_test_split(inputs, targets, test_size=0.2, random_state=42)

batch_size = 80
no_epochs = 1000

# defining the optimizer
tf.keras.optimizers.Adam(
    learning_rate=0.1, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=True,
    name='Adam'
)

#defining model parameters
node=36
alfa=0.001
initializer = tf.keras.initializers.GlorotNormal(seed=None)

#construction of model with 5 hidden layers
model = Sequential([
    (Dense(node, input_dim=n_feature, kernel_initializer=initializer, 
           activation='relu', kernel_regularizer=regularizers.l2(alfa))),
    (Dense(node, kernel_initializer=initializer, activation='relu',
           kernel_regularizer=regularizers.l2(alfa))),
    (Dense(node, kernel_initializer=initializer, activation='relu',
           kernel_regularizer=regularizers.l2(alfa))), 
    (Dense(node, kernel_initializer=initializer, activation='relu',
           kernel_regularizer=regularizers.l2(alfa))), 
    (Dense(node, kernel_initializer=initializer, activation='relu',
           kernel_regularizer=regularizers.l2(alfa))), 
    (Dense(1,  kernel_initializer=initializer, activation='linear')) 
])


model.compile(loss='mse',
            optimizer= 'Adam',
            metrics=['mse'])

# training of the data:
history = model.fit(X_train, Y_train,
          batch_size=batch_size,
          epochs=no_epochs,
          verbose=0)


# Performance evaluation w/ test data:
score = model.evaluate(X_test, Y_test, verbose=0)

print(f'> MSE: {score}')

In [ ]:
# predictions:
test_predictions = model.predict(X_test).flatten()
train_predictions = model.predict(X_train).flatten()

In [ ]:
score_test = model.evaluate(X_test, Y_test,  verbose=0)
r2_test = r2_score(Y_test, test_predictions)

score_train = model.evaluate(X_train, Y_train,  verbose=0)
r2_train = r2_score(Y_train, train_predictions)

print(f'> test: MSE: {score_test[0]}, R2: {r2_test}')
print(f'> train: MSE: {score_train[0]}, R2: {r2_train}')

In [ ]:
#Transformed values
a = plt.axes(aspect='equal')
plt.scatter(Y_train, train_predictions, c='#9467bd')

plt.scatter(Y_test, test_predictions, c='#8c564b')

plt.xlabel('True ')
plt.ylabel('Predictions ')
lims = [np.min(targets)*0.9-1,np.max(targets)*1.1]
plt.xlim(lims)
plt.ylim(lims)

_ = plt.plot(lims, lims)

In [ ]:
#Untransformed values
a = plt.axes(aspect='equal')
plt.scatter(Y_train**2, train_predictions**2, c='#9467bd')

plt.scatter(Y_test**2, test_predictions**2, c='#8c564b')

plt.xlabel('True ')
plt.ylabel('Predictions ')
lims = [(np.min(targets)**2)*0.9-1,(np.max(targets)**2)*1.1]
plt.xlim(lims)
plt.ylim(lims)

_ = plt.plot(lims, lims)

In [ ]:
# MSEs of untransformed data (in degree celcius)
loss_test = np.sqrt(mean_squared_error(test_predictions**2,Y_test**2))
loss_train = np.sqrt(mean_squared_error(train_predictions**2, Y_train**2))
print(loss_test, loss_train)

In [ ]:
# Concating of results:
result_train=pd.concat([pd.DataFrame(scaler.inverse_transform(X_train)).reset_index(drop=True),
                        pd.DataFrame(train_predictions**2).reset_index(drop=True), 
                        pd.DataFrame(Y_train**2).reset_index(drop=True)], axis=1, ignore_index=True)
result_test=pd.concat([pd.DataFrame(scaler.inverse_transform(X_test)).reset_index(drop=True),
                       pd.DataFrame(test_predictions**2).reset_index(drop=True), 
                       pd.DataFrame(Y_test**2).reset_index(drop=True)], axis=1, ignore_index=True)
results = pd.concat([result_test, result_train], axis=0, ignore_index=True)
results

In [ ]:
results.to_csv (r'NiTiHf_NN_result.csv', index = False, header=True,sep=',', decimal='.')

In [ ]:
# For designing of new alloys, you may use this command: new_predictions = model.predict(new_alloy_dataset)

In [ ]:
X_test.shape

# Feature Selection

In [ ]:
batch_size =80
no_epochs = 1000

# K-fold Cross Validation 
num_folds = 5 

subset_saves=[]

for subset in range (16,7,-1):    
    inputs_sub = np.array(inputs[:,0:subset])

    kfold = KFold(n_splits=num_folds, shuffle=True)
    fold_no = 1
    j=0
    scores=[]
    mse_per_fold = []
    train_mse_per_fold = []
    tf.keras.optimizers.Adam(
        learning_rate=0.03, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=True,
        name='Adam'
    )
    for train_index, test_index in kfold.split(inputs):  # training of data in 5 folds 

        X_train, X_test = inputs_sub[train_index],  inputs_sub[test_index]
        Y_train, Y_test = targets[train_index], targets[test_index]
        
        initializer = tf.keras.initializers.GlorotNormal(seed=None)
        
        node = 30
        reg_parameter = 0.001

        model_sub = Sequential([
            (Dense(node, input_dim=subset, kernel_initializer=initializer, activation='relu',
                   kernel_regularizer=regularizers.l2(reg_parameter))),
            (Dense(node, kernel_initializer=initializer, activation='relu',
                   kernel_regularizer=regularizers.l2(reg_parameter))),
            (Dense(node, kernel_initializer=initializer, activation='relu',
                   kernel_regularizer=regularizers.l2(reg_parameter))),
            (Dense(node, kernel_initializer=initializer, activation='relu',
                   kernel_regularizer=regularizers.l2(reg_parameter))),
            (Dense(node, kernel_initializer=initializer, activation='relu',
                   kernel_regularizer=regularizers.l2(reg_parameter))),
            (Dense(1, kernel_initializer=initializer, activation='linear'))
            ])

        model_sub.compile(loss='mse',
                optimizer='adam',
                metrics=['mse'])

        history = model_sub.fit(X_train, Y_train,
              batch_size=32,
              epochs=no_epochs,
              verbose=0)

        scores.append(model_sub.evaluate(X_test, Y_test, verbose=0))
        mse_per_fold.append(scores[j][:1])

        trainmse = model_sub.evaluate(X_train, Y_train,  verbose=0)
        train_mse_per_fold.append(trainmse)

        fold_no = fold_no + 1
        j = j + 1

        fold_no=1
       
    meanscore=np.mean(mse_per_fold) # average mse of 5 folds for selected subset
    meanscore_train=np.mean(train_mse_per_fold)

    subset_saves.append([subset, meanscore, meanscore_train]) 


In [ ]:
subset_saves = pd.DataFrame(subset_saves, columns = ('# of features', 'test score', 'train score'))
subset_saves

In [ ]:
subset_saves.to_csv (r'NN_subset_selection.csv', index = False, header=True,sep=',', decimal='.')

# Optimization of hyperparameters:

In [ ]:
batch_size =80
no_epochs = 1000

# K-fold Cross Validation w/ 5 folds
num_folds = 5
msesaves=[]

limit=10000

for nnode in range(15,48,6): 
    for alpha in range (1,11,3): 
        alphan=alpha/1000 
        kfold = KFold(n_splits=num_folds, shuffle=True)
        fold_no = 1
        j=0
        scores=[]
        mse_per_fold = []
        train_mse_per_fold = []
        tf.keras.optimizers.Adam(
            learning_rate=0.03, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=True,
            name='Adam'
        )
        for train_index, test_index in kfold.split(inputs):
            
            X_train, X_test = inputs[train_index],  inputs[test_index]
            Y_train, Y_test = targets[train_index], targets[test_index]
                
            initializer = tf.keras.initializers.GlorotNormal(seed=None)

            model_opt = Sequential([
                (Dense(nnode, input_dim=n_feature, kernel_initializer=initializer, activation='relu',
                       kernel_regularizer=regularizers.l2(alphan))),
                (Dense(nnode, kernel_initializer=initializer, activation='relu',
                       kernel_regularizer=regularizers.l2(alphan))),
                (Dense(nnode, kernel_initializer=initializer, activation='relu',
                       kernel_regularizer=regularizers.l2(alphan))),
                (Dense(nnode, kernel_initializer=initializer, activation='relu',
                       kernel_regularizer=regularizers.l2(alphan))),
                (Dense(nnode, kernel_initializer=initializer, activation='relu',
                       kernel_regularizer=regularizers.l2(alphan))),
                (Dense(1, kernel_initializer=initializer, activation='linear'))
                ])

            
            model_opt.compile(loss='mse',
                    optimizer='adam',
                    metrics=['mse'])

            
            history = model_opt.fit(X_train, Y_train,
                  batch_size=32,
                  epochs=no_epochs,
                  verbose=0)

            
            scores.append(model_opt.evaluate(X_test, Y_test, verbose=0))
            mse_per_fold.append(scores[j][:1])
                        
            trainmse = model_opt.evaluate(X_train, Y_train,  verbose=0)
            train_mse_per_fold.append(trainmse)

           
            fold_no = fold_no + 1
            j = j + 1

            fold_no=1
            


        meanscore=np.mean(mse_per_fold)
        meanscore_train=np.mean(train_mse_per_fold)

        msesaves.append([alphan, nnode, meanscore, meanscore_train])



        if meanscore < limit:
            nnode_op=nnode
            alpha_op=alphan
            limit=meanscore

In [ ]:
msesaves = pd.DataFrame(msesaves, columns = ('alpha','# of nodes', 'test score', 'train score'))
msesaves

In [ ]:
msesaves.to_csv (r'NN_optimization.csv', index = False, header=True,sep=',', decimal='.')